In [4]:
import pandas as pd
from ruspy.estimation.estimation import estimate

In [9]:
# Visit the documenation for the exact keys necessary or possible.
init_dict_estimation = {
    "model_specifications": {
        "discount_factor": 0.9999,
        "number_states": 90,
        "maint_cost_func": "linear",
        "cost_scale": 1e-3,
    },
    "optimizer": {
        "approach": "NFXP",
        "algorithm": "scipy_L-BFGS-B",
        "gradient": "Yes",
    },
}

In [10]:
# In this demonstration setting, only the data of group 4 is included. For different data sets, either prepare
# yourself from raw data (e.g. https://editorialexpress.com/jrust/research.html) or visit the data repository
# of the OpenSourceCommunity for Rust's (1987) data (https://github.com/OpenSourceEconomics/zurcher-data).
data = pd.read_pickle("group_4.pkl")

In [11]:
result_transitions, result_fixp = estimate(init_dict_estimation, data)

In [12]:
result_fixp

{'x': array([10.07495174,  2.29309306]),
 'fun': 163.58428337702918,
 'status': 'success',
 'message': b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH',
 'jac': array([ 2.01342507e-05, -4.10620399e-05])}

In [13]:
result_transitions

{'trans_count': array([1682, 2555,   55], dtype=int64),
 'x': array([0.39189229, 0.59529346, 0.01281426]),
 'fun': 3140.5705571081444}